# Behavior cloning flow

Create the full flow for training a model for behavior cloning which is completely separated from the other ones. 

In [1]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pprint
import socket
import pathlib
import yaml
import tqdm
import papermill
# from automate import automate_exprun
import bc_factory 
from demonstration.demonstration import list_demos

In [ ]:
# Setting up a separate directory for generated and computed data
exprun_path, result_path = bc_factory.external_setup("BerryPicker-BC")

Hostname is glassy
Path for external experiments: C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun
Path for external data: C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\result
***ExpRun**: Loading pointer config file:
	C:\Users\lotzi\.config\BerryPicker\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	c:\Users\lotzi\Work\_Config\BerryPicker\cfg\settings.yaml
***ExpRun**: Experiment config path changed to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun
***ExpRun**: Experiment data path changed to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\result
***ExpRun**: Experiment demonstration copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\demonstration
***ExpRun**: Experiment sensorprocessing_conv_vae copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\sensorprocessing_conv_vae
***ExpRun**: Experiment robot_al5d copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\robot_al5d
***ExpRun**: Experiment automate

In [3]:
def generate_sensorprocessing_conv_vae(exprun_path, result_path, params):
    """Generate the experiment for the conv-vae sensorprocessing with the right training data and parameters. Returns the experiment and runname"""
    val = {}
    val["latent_size"] = params["latent_size"]
    val["epochs"] = params["epochs"]
    val["save_period"] = 5
    val["training_data"] = params["training_data"]
    val["validation_data"] = params["validation_data"]
    # save the generated exprun spec
    path = pathlib.Path(Config().get_experiment_path(), params["exp_sp"], params["run_sp"] + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    # now, generate the entry in the automation file 
    v = {}
    v["name"] = "TrainVAE"
    v["notebook"] = "sensorprocessing/Train-Conv-VAE.ipynb"
    vparams = {}
    vparams["exp"] = params["exp_sp"]
    vparams["run"] = params["run_sp"]
    vparams["external_path"] = exprun_path.as_posix()
    vparams["data_path"] = result_path.as_posix()
    v["params"] = vparams

    return {"experiment": params["exp_sp"], "runname": params["run_sp"], "automation_entry": v}

In [4]:
def generate_behaviorcloning(exprun_path, result_path, params):
    """Generate the experiment for the behavior cloning with the right training data and parameters. At the same time, it also generates the corresponding bcrun experiment, which will be used to drive the robot.  
    FIXME there will be parameters etc
    For the time being this is generating the mdn type
    """    
    val = {}
    val["exp_sp"] = params["exp_sp"]
    val["run_sp"] = params["run_sp"]
    # these had not yet been studied for alternatives
    val["sequence_length"] = params["sequence_length"]
    val["loss"] = params["loss"]
    val["latent_size"] = params["latent_size"]
    val["epochs"] = params["epochs"]

    if params["algorithm"] == "lstm-mdn": 
        val["exp_mdn"] = "behavior_cloning"
        val["run_mdn"] = "mdn_for_bc_00"
        val["controller"] = "bc_LSTM_MDN"
        val["controller_file"] = "controller.pth"
        val["hidden_size"] = params["hidden_size"]
    elif params["algorithm"] == "lstm":
        val["controller"] = "bc_LSTM"
        val["controller_file"] = "controller.pth"
        val["num_layers"] = params["num_layers"]
        val["hidden_size"] = params["hidden_size"]
    elif params["algorithm"] == "lstm-residual":
        val["controller"] = "bc_LSTM_Residual"
        val["controller_file"] = "controller.pth"
        val["hidden_size"] = params["hidden_size"]
    elif params["algorithm"] == "mlp":
        val["controller"] = "bc_MLP"
        val["controller_file"] = "controller.pth"
        val["hidden_layers"] = params["hidden_layers"]
        val["hidden_layer_1"] = params["hidden_layer_1"]
        val["hidden_layer_2"] = params["hidden_layer_2"]
        
    else:
        raise Exception(f"*** generate_behaviorcloning: algorithm {params['algorithm']} not yet supported")
    
    val["control_size"] = 6 
    val["training_data"] = params["training_data"]
    val["validation_data"] = params["validation_data"]

    # save the generated exprun spec
    path = pathlib.Path(Config().get_experiment_path(), params["exp_bc"], params["run_bc"] + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    # now, generate the entry in the automation file 
    v = {}
    v["name"] = "BehaviorCloning"
    v["notebook"] = "behavior_cloning/Train_BehaviorCloning.ipynb"
    vparams = {}
    vparams["run"] = params["run_bc"]
    vparams["exp"] = params["exp_bc"]
    vparams["external_path"] = exprun_path.as_posix()
    vparams["data_path"] = result_path.as_posix()
    v["params"] = vparams

    # now, generate the bcrun experiment file, which is the same with the "runbc_" prefix
    # FIXME: these need to be configurable
    valrun = {}
    valrun["name"] = "generated_run_{params['run_bc']}"
    valrun["exp_robot_controller"] = "robot_al5d"
    valrun["run_robot_controller"] = "position_controller_00"
    valrun["exp_camera_controller"] = "controllers"
    valrun["run_camera_controller"] = "camera_cam0_controller"
    valrun["control_camera"] = "dev0"
    valrun["exp_bc"] = "behavior_cloning"
    valrun["run_bc"] = "bc_lstm_0001"

    path = pathlib.Path(Config().get_experiment_path(), params["exp_bc"], "runbc_"+ params["run_bc"] + ".yaml")
    with open(path, "w") as f:
        yaml.dump(valrun, f)

    return {"experiment": params["exp_bc"], "runname": params["run_bc"], "automation_entry": v}

### Generate a range of exp/runs to be run

In [ ]:
expruns = []

# overall values
latent_size = 128

# FIXME: set the training data through some data structure passed as parameter....
# setting the training data
cam = "dev0"
run_demo = "touch-apple"
experiment = "demonstration"
# run = "freeform"
# run = "random-both-cameras"
run = "random-both-cameras-video"
exp = Config().get_experiment(experiment, run_demo)
demos = list_demos(exp)
print(demos)

data = []
for demo in demos:
    data.append([run_demo, demo, cam])

training_data = data[0:-2]
sp_training_data = training_data
bc_training_data = training_data

# setting the validation data
validation_data = data[-2:]
sp_validation_data = validation_data
bc_validation_data = validation_data

# determine these values based on experience
epochs_sp = 300
epochs_bc = 500

# generate sensor processing expruns
exp_sp = "sensorprocessing_conv_vae"
run_sp = "sp_conv_vae_0001"
params = {}
params["exp_sp"] = exp_sp
params["run_sp"] = run_sp
params["latent_size"] = latent_size
params["epochs"] = epochs_sp
params["training_data"] = sp_training_data
params["validation_data"] = sp_validation_data
exprun = generate_sensorprocessing_conv_vae(
    exprun_path = exprun_path, result_path = result_path, params = params)
expruns.append(exprun)

# generate the behavior cloning expruns to match with the sensor processing components from above

exp_bc = "behavior_cloning"

# an MLP model
params = {}
params["exp_bc"] = exp_bc
params["run_bc"] = "bc_mlp_0001"
params["algorithm"] = "mlp"
params["exp_sp"] = exp_sp
params["run_sp"] = run_sp
params["latent_size"] = latent_size
params["hidden_layers"] = 2
params["hidden_layer_1"] = 50
params["hidden_layer_2"] = 20
params["epochs"] = epochs_bc
params["sequence_length"] = 1
params["loss"] = "MSELoss"
params["training_data"] = bc_training_data
params["validation_data"] = bc_validation_data
exprun = generate_behaviorcloning(exprun_path=exprun_path, result_path=result_path, params=params)
expruns.append(exprun)

# a pure lstm model
params = {}
params["exp_bc"] = exp_bc
params["run_bc"] = "bc_lstm_0001"
params["algorithm"] = "lstm"
params["exp_sp"] = exp_sp
params["run_sp"] = run_sp
params["latent_size"] = latent_size
params["hidden_size"] = 32
params["num_layers"] = 2
params["epochs"] = epochs_bc
params["sequence_length"] = 10
params["loss"] = "MSELoss"
params["training_data"] = bc_training_data
params["validation_data"] = bc_validation_data
exprun = generate_behaviorcloning(exprun_path=exprun_path, result_path=result_path, params=params)
expruns.append(exprun)

# a residual lstm model
params = {}
params["exp_bc"] = exp_bc
params["run_bc"] = "bc_lstm_residual_0001"
params["algorithm"] = "lstm-residual"
params["exp_sp"] = exp_sp
params["run_sp"] = run_sp
params["latent_size"] = latent_size
params["hidden_size"] = 32
params["epochs"] = epochs_bc
params["sequence_length"] = 10
params["loss"] = "MSELoss"
params["training_data"] = bc_training_data
params["validation_data"] = bc_validation_data
exprun = generate_behaviorcloning(exprun_path=exprun_path, result_path=result_path, params=params)
expruns.append(exprun)


# an lstm-mdn exp
params = {}
params["exp_bc"] = exp_bc
params["run_bc"] = "bc_lstm_mdn_0001"
params["algorithm"] = "lstm-mdn"
params["exp_sp"] = exp_sp
params["run_sp"] = run_sp
params["latent_size"] = latent_size
params["hidden_size"] = 32
params["epochs"] = epochs_bc
params["sequence_length"] = 10
params["loss"] = "MSELoss"
params["training_data"] = bc_training_data
params["validation_data"] = bc_validation_data
exprun = generate_behaviorcloning(exprun_path=exprun_path, result_path=result_path, params=params)
expruns.append(exprun)


***ExpRun**: Experiment default config C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\demonstration\_defaults_demonstration.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: demonstration/touch-apple successfully loaded
['2025_08_07__15_13_18', '2025_08_07__15_17_27', '2025_08_07__15_18_27', '2025_08_07__15_19_25', '2025_08_07__15_20_47', '2025_08_07__15_22_29', '2025_08_07__15_23_23', '2025_08_07__15_24_25', '2025_08_07__15_25_18', '2025_08_07__15_26_10', '2025_08_07__15_28_02']


### Generate an automation script. 

* FIXME: the notebooks should have support to set the exprun and results directories from the automation exp. (This seems to actually exist)


In [6]:
value = {}
val = []
value["exps_to_run"] = val

creation_style = "exist-ok"
# creation_style = "discard-old"

for exprun in expruns:
    v = exprun["automation_entry"]
    v["params"]["creation_style"] = creation_style
    val.append(v)

path = pathlib.Path(Config().get_experiment_path(), "automate", "flow_bc.yaml")
with open(path, "w") as f:
    yaml.dump(value, f)

### Run the automation script
* FIXME: the existing automation does not take into consideration the external directories the right way. I am trying to fix this. 
* FIXME: this should be done such that I can also run it from command line

In [7]:
experiment = "automate"
run = "flow_bc"
exp = Config().get_experiment(experiment, run)

for item in tqdm.tqdm(exp["exps_to_run"]):
    print(f"***Automating {item['name']}")
    notebook_path = pathlib.Path("..", item["notebook"])
    output_filename = f"{notebook_path.stem}_{item['params']['exp']}_{item['params']['run']}_output{notebook_path.suffix}"
    # FIXME: this should go into the data dir of the appropriate exp... but I don't have access to it... 
    pprint.pprint(item["params"])
    output_path = pathlib.Path(result_path, output_filename)
    try:
        papermill.execute_notebook(
            notebook_path,
            output_path.absolute(),
            cwd=notebook_path.parent,
            parameters=item["params"]
        )
    except Exception as e:
        print(f"There was an exception {e}")    

***ExpRun**: Experiment default config C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\automate\_defaults_automate.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: automate/flow_bc successfully loaded


  0%|          | 0/5 [00:00<?, ?it/s]Passed unknown parameter: exp
c:\Users\lotzi\Work\_VirtualEnvs\BerryPicker\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


***Automating TrainVAE
{'creation_style': 'exist-ok',
 'data_path': 'C:/Users/lotzi/Work/_DataExternal/BerryPicker-BC/result',
 'exp': 'sensorprocessing_conv_vae',
 'external_path': 'C:/Users/lotzi/Work/_DataExternal/BerryPicker-BC/exprun',
 'run': 'sp_conv_vae_0001'}


 20%|██        | 1/5 [00:03<00:12,  3.19s/it]Passed unknown parameter: exp


***Automating BehaviorCloning
{'creation_style': 'exist-ok',
 'data_path': 'C:/Users/lotzi/Work/_DataExternal/BerryPicker-BC/result',
 'exp': 'behavior_cloning',
 'external_path': 'C:/Users/lotzi/Work/_DataExternal/BerryPicker-BC/exprun',
 'run': 'bc_mlp_0001'}


...

 60%|██████    | 3/5 [00:10<00:07,  3.65s/it]Passed unknown parameter: exp


***Automating BehaviorCloning
{'creation_style': 'exist-ok',
 'data_path': 'C:/Users/lotzi/Work/_DataExternal/BerryPicker-BC/result',
 'exp': 'behavior_cloning',
 'external_path': 'C:/Users/lotzi/Work/_DataExternal/BerryPicker-BC/exprun',
 'run': 'bc_lstm_residual_0001'}


 80%|████████  | 4/5 [00:14<00:03,  3.81s/it]Passed unknown parameter: exp


***Automating BehaviorCloning
{'creation_style': 'exist-ok',
 'data_path': 'C:/Users/lotzi/Work/_DataExternal/BerryPicker-BC/result',
 'exp': 'behavior_cloning',
 'external_path': 'C:/Users/lotzi/Work/_DataExternal/BerryPicker-BC/exprun',
 'run': 'bc_lstm_mdn_0001'}


100%|██████████| 5/5 [00:21<00:00,  4.26s/it]
